In [1]:
import json
import pandas as pd
import numpy as np

In [3]:
with open("oxford_sensewords.json", "r") as f:
    oxford_sensewords = json.load(f)

In [4]:
with open("oxford_sensewords_meta.json", "r") as f:
    oxford_sensewords_meta = json.load(f)

In [5]:
len(list(oxford_sensewords.keys())), len(list(oxford_sensewords_meta.keys()))

(200, 200)

In [6]:
overall_words = 0 
for key in oxford_sensewords.keys():
    overall_words += len(oxford_sensewords[key])
overall_words

1010

In [7]:
overall_words = 0 
for key in oxford_sensewords_meta.keys():
    overall_words += len(oxford_sensewords_meta[key])
overall_words

1010

In [8]:
oxford_sensewords['word']

['mot', 'bruit [masculine] qui court', 'parole', 'formuler', 'nouvelles']

In [9]:
oxford_sensewords_meta['word']

['mot', 'bruit [masculine] qui court', 'parole', 'formuler', 'nouvelles']

In [10]:
df = pd.read_csv("naive_transaltion_en_ru_fr_LEMMAS_WHOLE_LIST.csv")
df.head()

,Unnamed: 0,word,local_word,examples,local_examples
0,0,for,pour,"['for the bank', 'for last month', 'for one ye...","['pour la banque', 'pour le mois dernier', 'pe..."
1,1,for,pour,['for god'],['pour dieu']
2,2,for,au cours de,[],[]
3,3,for,à l'égard de,[],[]
4,4,for,à des fins,[],[]


In [24]:
yandex_naive_sensewords = {}
for ind in range(len(df)):
    df_el = list(df.iloc[ind])
    #print(list(df.iloc[ind]))
    #print(df_el[1],df_el[2])
    if df_el[1] in yandex_naive_sensewords:
        yandex_naive_sensewords[df_el[1].lower()].append(df_el[2].lower())
    else:
        yandex_naive_sensewords[df_el[1].lower()] = []
        yandex_naive_sensewords[df_el[1].lower()].append(df_el[2].lower())
    

In [30]:
yandex_naive_sensewords['america']

['amérique']

In [28]:
oxford_sensewords['America']

[]

In [14]:
def get_intersection(oxford_senses, compared_senses):
    detected_words_oxford = []
    detcted_words_compared = []
    for ox_word in oxford_senses:
        for comp_word in compared_senses:
            if ox_word == comp_word or ox_word in comp_word or comp_word in ox_word:
                detected_words_oxford.append(ox_word)
                detcted_words_compared.append(comp_word)
    oxford_set = set(oxford_senses)
    compared_set = set(compared_senses)
    intersect_oxford = oxford_set.intersection(set(detected_words_oxford))
    if (len(set(oxford_senses))) == 0:
        oxford_senses = ['dummy_word']
    oxford_recall = len(intersect_oxford) / (len(set(oxford_senses)))
    intersect_compared = compared_set.intersection(set(detcted_words_compared))
    new_words_from_compare = (len(compared_set) - len(intersect_compared)) / (len(set(oxford_senses)))
    print(intersect_oxford, intersect_compared)
    return (oxford_recall + new_words_from_compare)/2

get_intersection(oxford_sensewords['word'] ,yandex_naive_sensewords['word'])


{'nouvelles', 'formuler', 'mot'} {'nouvelle', 'formuler', 'mot'}


0.6

In [32]:
def sense_words_dicts_comparison(oxford, comparison_dict):
    word = []
    oxford_sensewords = []
    compared_sensewords = []
    comparison_value = []
    for ox_key in oxford.keys(): 
        word.append(ox_key)
        oxford_sensewords.append(oxford[ox_key])
        compared_sensewords.append(comparison_dict[ox_key.lower()])
        #print (ox_key, oxford[ox_key],comparison_dict[ox_key])
        #print()
        comp_res = get_intersection(oxford[ox_key],comparison_dict[ox_key.lower()])
        comparison_value.append(comp_res)
    data = pd.DataFrame(list(zip(word,oxford_sensewords,compared_sensewords, comparison_value)),columns =['word', 'oxford_sensewords', 'compared_sensewords','comparison_value'])
    return data
comp_df = sense_words_dicts_comparison(oxford_sensewords_meta ,yandex_naive_sensewords)        
        

{'nouvelles', 'formuler', 'mot'} {'nouvelle', 'formuler', 'mot'}
{'le (before a consonant) / l’ (before a vowel)', 'les'} {'le'}
{'et', '(et)', '(et même)'} {'et', 'donc et', 'et aussi', 'et en outre'}
{'que', 'tellement', 'ce (before a consonant) / c’ (before a vowel)', 'que (before a consonant) / qu’ (before a vowel)', 'cette … (là)', 'cela / ça', 'ce (before a consonant) / cet (before a vowel)', 'qui', 'ce (before a consonant) / cet (before a vowel) … (là)', 'celui/celle-là', 'cette', 'comme…'} {'comme', 'que', 'afin que', 'ce', 'tel', 'qui'}
{'car', 'pour'} {'car', 'pour'}
{'de', '(avec)', 'avec'} {'en compagnie de', 'avec'}
{'mais', 'sauf'} {'mais', 'sauf'}
{'tu/toi/te', 'tu/vous', 'vous'} {'tu', 'vous'}
{'cette … ci', 'ce (before a consonant) / cet (before a vowel)', 'ce (before a consonant…', 'ce (before a consonant) / cet (before a vowel) … (ci)', 'cette', 'ce (before a consonant) / cet (before a vowel) / cette'} {'ce'}
{'ne … pas', 'n’est-ce pas', 'ne…pas', 'pas'} {'pas'}
{'av

In [33]:
comp_df.head(20)

,word,oxford_sensewords,compared_sensewords,comparison_value
0,word,"[mot, bruit [masculine] qui court, parole, for...","[mot, nouvelle, mot, dire, formuler, textuel, ...",0.600000
1,the,"[les, la (before a consonant) / l’…, le (befor...","[ce, le, le, tant]",0.500000
2,and,"[(et), (indique une intention ou un but), plus...","[et, et aussi, et en outre, donc et, avec]",0.333333
3,that,[que (before a consonant) / qu’ (before a vowe...,"[que, comme, afin que, qui, ce, propre, donc, ...",0.538462
4,for,"[en, en faveur de, malgré, pour, en raison de,...","[pour, pour, au cours de, à l'égard de, à des ...",0.312500
5,with,"[(avec), contre, pour, en + verbe au participe...","[avec, et en outre, en compagnie de]",0.222222
6,but,"[mais, excepté, sauf]","[mais, et, cependant, seulement, sauf]",0.833333
7,you,"[tu/toi/te, espèce de, on, tu/vous, vous]","[vous, tu]",0.300000
8,this,"[cette, ce (before a consonant) / cet (before ...","[ce, ce, actuel, ce, donc]",0.500000
9,not,"[ne…pas, pas, non, ne … pas, n’est-ce pas]","[pas, pas]",0.400000


In [34]:
comp_df.to_csv("yandexNaive_vs_cambridge_comparison_WHOLE_LIST.csv")

In [35]:
np.mean(list(comp_df['comparison_value'])), np.median(list(comp_df['comparison_value']))

(0.9171662376235585, 0.6666666666666666)